In [ ]:
%store

In [1]:
%store -r __importRegression

In [2]:
__importRegression

/home/maviator/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [49]:
endMonth = 33
fnameTest = '../input/validation/test_' + str(endMonth) + '.csv'
fnameTrain = '../input/validation/train_' + str(endMonth) + '.csv'
train = pd.read_csv(fnameTrain)
test = pd.read_csv(fnameTest)
#train = pd.read_csv('../input/train.csv')
#test = pd.read_csv('../input/test.csv')
items = pd.read_csv("../input/items.csv")
categories = pd.read_csv("../input/item_categories.csv")

print (train.shape, test.shape, items.shape, categories.shape)

((2882335, 6), (31531, 3), (22170, 3), (84, 2))


In [50]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [51]:
test.head()

,shop_id,item_id,item_cnt_day
0,2,31,1.0
1,2,486,3.0
2,2,787,1.0
3,2,794,1.0
4,2,968,1.0


## Extract shop total sales

In [52]:
train_shop = train.copy()

In [53]:
train_shop.drop(['date', 'date_block_num', 'item_price', 'item_id'], axis=1, inplace=True)

In [54]:
train_shop.head()

,shop_id,item_cnt_day
0,59,1.0
1,25,1.0
2,25,-1.0
3,25,1.0
4,25,1.0


In [55]:
train_shop = train_shop.groupby(['shop_id'], as_index=False).agg({'item_cnt_day':sum})

In [56]:
train_shop.head()

,shop_id,item_cnt_day
0,0,11705.0
1,1,6311.0
2,2,29893.0
3,3,27742.0
4,4,43111.0


In [57]:
train_shop['shop_total_sales'] = train_shop.item_cnt_day

In [58]:
train_shop.drop('item_cnt_day', axis=1, inplace=True)

In [59]:
train_shop.head()

,shop_id,shop_total_sales
0,0,11705.0
1,1,6311.0
2,2,29893.0
3,3,27742.0
4,4,43111.0


## Extract item total sales

In [60]:
train_items = train.copy()

In [61]:
train_items.drop(['date', 'date_block_num', 'shop_id', 'item_price'], axis=1, inplace=True)

In [62]:
train_items.head()

,item_id,item_cnt_day
0,22154,1.0
1,2552,1.0
2,2552,-1.0
3,2554,1.0
4,2555,1.0


In [63]:
train_items = train_items.groupby(['item_id'], as_index=False).agg({'item_cnt_day':sum})

In [64]:
train_items['item_total_sales'] = train_items.item_cnt_day
train_items.drop('item_cnt_day', axis=1, inplace=True)

In [65]:
train_items.head()

,item_id,item_total_sales
0,0,1.0
1,1,6.0
2,2,2.0
3,3,2.0
4,4,1.0


In [66]:
train.pop('date')
train.pop('date_block_num')
train.pop('item_price');

In [67]:
train.head()

,shop_id,item_id,item_cnt_day
0,59,22154,1.0
1,25,2552,1.0
2,25,2552,-1.0
3,25,2554,1.0
4,25,2555,1.0


In [68]:
train = train.groupby(['shop_id', 'item_id'], as_index=False).agg({'item_cnt_day':sum})

In [69]:
print train.shape
train.head()

(417004, 3)


,shop_id,item_id,item_cnt_day
0,0,30,31.0
1,0,31,11.0
2,0,32,16.0
3,0,33,6.0
4,0,35,15.0


In [70]:
train.item_cnt_day.describe()

count    417004.000000
mean          8.578215
std          63.342855
min         -12.000000
25%           1.000000
50%           3.000000
75%           7.000000
max       19503.000000
Name: item_cnt_day, dtype: float64

Take average sales in 33 months and remove duplicates

In [71]:
train.item_cnt_day = train.item_cnt_day / (endMonth-1)

In [72]:
train.head()

,shop_id,item_id,item_cnt_day
0,0,30,0.96875
1,0,31,0.34375
2,0,32,0.50000
3,0,33,0.18750
4,0,35,0.46875


In [73]:
train.drop_duplicates(inplace=True)
train.shape

(417004, 3)

In [74]:
print test.shape
test.head()

(31531, 3)


,shop_id,item_id,item_cnt_day
0,2,31,1.0
1,2,486,3.0
2,2,787,1.0
3,2,794,1.0
4,2,968,1.0


## Merge shop_total_sales and item_total_sales

In [75]:
train = pd.merge(train, train_shop, on='shop_id', sort=False, how='left')
test = pd.merge(test, train_shop, on='shop_id', sort=False, how='left')

In [76]:
train = pd.merge(train, train_items, on='item_id', sort=False, how='left')
test = pd.merge(test, train_items, on='item_id', sort=False, how='left')

In [77]:
train.head()

,shop_id,item_id,item_cnt_day,shop_total_sales,item_total_sales
0,0,30,0.96875,11705.0,2088.0
1,0,31,0.34375,11705.0,1417.0
2,0,32,0.50000,11705.0,2070.0
3,0,33,0.18750,11705.0,820.0
4,0,35,0.46875,11705.0,222.0


In [78]:
test.head()

,shop_id,item_id,item_cnt_day,shop_total_sales,item_total_sales
0,2,31,1.0,29893.0,1417.0
1,2,486,3.0,29893.0,1635.0
2,2,787,1.0,29893.0,253.0
3,2,794,1.0,29893.0,17.0
4,2,968,1.0,29893.0,660.0


In [79]:
test.fillna(0)

,shop_id,item_id,item_cnt_day,shop_total_sales,item_total_sales
0,2,31,1.0,29893.0,1417.0
1,2,486,3.0,29893.0,1635.0
2,2,787,1.0,29893.0,253.0
3,2,794,1.0,29893.0,17.0
4,2,968,1.0,29893.0,660.0
5,2,988,1.0,29893.0,4.0
6,2,1075,1.0,29893.0,983.0
7,2,1121,1.0,29893.0,27.0
8,2,1377,1.0,29893.0,530.0
9,2,1387,1.0,29893.0,691.0


In [80]:
print (train.shape, test.shape)

((417004, 5), (31531, 5))


In [81]:
train_labels = train.pop('item_cnt_day')

In [82]:
labels = test.pop('item_cnt_day')

test_id = test.pop('ID')

# LightGBM tunning

LBC = LGBMRegressor()
lb_param_grid = {
              'n_estimators' : [10, 50, 100],
              'learning_rate': [0.1, 0.01],
              'max_depth': [4, 8]
              }

lGBC = GridSearchCV(LBC,param_grid = lb_param_grid, cv=5,
                     scoring="neg_mean_squared_error", n_jobs= -1, verbose = 1)

lGBC.fit(train, train_labels)

lgbm = lGBC.best_estimator_

# Best score
lGBC.best_score_

In [94]:
lgbm = LGBMRegressor()
lgbm.fit(train, train_labels)

LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0.0, n_estimators=10, n_jobs=-1, num_leaves=31,
       objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=50000,
       subsample_freq=1)

In [97]:
# Train RMSE error
train_predictions = lgbm.predict(train)
predictions = lgbm.predict(test)

print np.sqrt(mean_squared_error(train_labels, train_predictions))
print r2_score(train_labels, train_predictions)

1.15198628731
0.661311580818


In [98]:

predictions.shape

(31531,)

In [99]:
labels.shape

(31531,)

Clipping predictions in the [0, 20] range

In [100]:
for i in range(len(predictions)):
    if predictions[i] > 20:
        predictions[i] = 20
    if predictions[i] < 0:
        predictions[i] = 20

for i in range(len(labels)):
    if labels[i] > 20:
        labels[i] = 20
    if labels[i] < 0:
        labels[i] = 20

Compute RMSE

In [103]:
print np.sqrt(mean_squared_error(labels, predictions))
print r2_score(labels, predictions)

14.3828109588
0.0205705751679


## Submission

test_submit = pd.DataFrame({'ID': test_id, 'item_cnt_month': predictions})
print test_submit.shape
test_submit.to_csv('lgbm_all_train_totalSales.csv', index=False)
test_submit.head()

test_submit.item_cnt_month.describe()